In [ ]:
import os, glob

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation

In [ ]:
%matplotlib notebook

In [ ]:
directory = 'ex4_circular_bodies4/'
csv_files = list()
for file in glob.glob(directory + "*.csv"):
    csv_files.append(file)
csv_files.sort()
csv_files

In [ ]:
class TrajectoryData(object):
    def __init__(self, filename, color):
        self.filename = filename
        self.color = color
        self.df = pd.read_csv(self.filename)
        

#colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5'][:len(csv_files)]
colors = ['#00f54e',  '#0052f5', '#00f5d8', '#4100f5', '#797a7a'][:len(csv_files)]

trajectories = [TrajectoryData(f, c) for f, c in zip(csv_files, colors)]

In [ ]:
# Plot the entire trajectory
fig, ax = plt.subplots(1, 1, figsize=(5,5))
lines, pts = list(), list()

for trj in trajectories:
    xs = trj.df['x0'].values
    ys = trj.df['x1'].values
    last_idx = trj.df.index[-1]
    ax.plot(xs, ys, c=trj.color, alpha=0.8)
    ax.scatter(x=xs[last_idx], y=ys[last_idx],
               s=trj.df['mass'][last_idx]*10, color=trj.color)

x_range = ax.get_xlim()
y_range = ax.get_ylim()
plt.show()

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(1, 1, figsize=(5,5))
lines, pts = list(), list()

for trj in trajectories:
    line, = ax.plot([], [], lw=1, 
                    color=trj.color, alpha=0.5,
                    zorder=-1)
    particle, = ax.plot([], [], 'o',
                        markersize=(trj.df['mass'][0])**(1/3)*3, 
                        color=trj.color,
                        zorder=2)
    lines.append(line)
    pts.append(particle)

# initialization function: plot the background of each frame
def init():
    ax.set_xlim(x_range)
    ax.set_ylim(y_range)
    for key, spine in ax.spines.items():
        spine.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])

    for particle in pts:
        particle.set_data([], [])
        
    for line in lines:
        line.set_data([], [])
        
    return lines, pts

# animation function.  This is called sequentially
def animate(i):
    for idx, trj in enumerate(trajectories):
        xs = trj.df['x0'].values[:i]
        ys = trj.df['x1'].values[:i]
        xf, yf = trj.df['x0'][i], trj.df['x1'][i]
        pts[idx].set_data([xf], [yf])
        lines[idx].set_data(xs, ys)

    return lines, pts


# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=last_idx, interval=1, blit=True)

plt.show()

In [ ]:
fps = int(last_idx / 30) # so movie is 30s
anim.save(directory + 'output.mp4', fps=fps, dpi=200, extra_args=['-vcodec', 'libx264'])